In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Задача: разработать сервис, который будет предсказывать стоимость домов, основываясь на истории предложений.

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from ast import literal_eval
from sklearn.preprocessing import (StandardScaler, MinMaxScaler)
from sklearn.model_selection import (train_test_split, KFold, StratifiedKFold)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, StackingRegressor)
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
import shap
from xgboost import XGBRegressor

from sklearn.metrics import f1_score
from sklearn.metrics import (mean_squared_error, mean_absolute_error)
from sklearn.metrics import mean_absolute_percentage_error

In [3]:
# fix the RANDOM_SEED value so that the experiments are reproducible:
RANDOM_SEED = 42

In [4]:
data = pd.read_csv('/kaggle/input/diplomdata/data.csv')
data.head()

In [5]:
data.columns

In [6]:
data.info()

In [7]:
data['target'].value_counts()

In [8]:
data['target'] = data['target'].apply(lambda x: str(x).replace('$',''))
data['target'] = data['target'].apply(lambda x: str(x).replace('+',''))
data['target'] = data['target'].apply(lambda x: str(x).replace(',',''))
data['target'] = data['target'].apply(lambda x: str(x).replace('/mo','000'))
data['target'] = data['target'].apply(lambda x: str(x).replace('1,215 - 1,437','1300000'))
data['target'] = data['target'].apply(lambda x: str(x).replace('nan','225000'))
data['target'] = data['target'].apply(lambda x: str(x).replace('1215 - 1437000','1437000'))

In [9]:
X2 = data.drop('target',axis = 1).fillna(0)
y2 = data['target'].fillna(0)
skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X2, y2):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train2, X_test2 = X2[X2.index.isin(train_index)], X2[X2.index.isin(test_index)]
    y_train2, y_test2 = y2[y2.index.isin(train_index)], y2[y2.index.isin(test_index)]
    
#     # For training, fit() is used
    model = CatBoostRegressor(iterations=300)
    model.fit(X_train2, y_train2,eval_set=(X_test2, y_test2), verbose=100, cat_features=['status', 'private pool', 'propertyType', 'street', 'baths', 'homeFacts', 'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'mls-id', 'PrivatePool', 'MlsId'])


#     # For MAPE metric (or any other), we need the predictions of the model
    y_pred2 = model.predict(X_test2)

    print(np.sqrt(mean_squared_error(y_test2, y_pred2)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test2, y_pred2))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test2, y_pred2))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test2, y_pred2))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")

The MAPE mertic for the default CatBoost model using 4-fold CV is: 15.44%.
The MAE mertic for default CatBoost model: 300391.57
The RMSE default CatBoost model: 1300406.41

In [10]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X2)

In [11]:
shap.summary_plot(shap_values, X2)

Посмотрю на количество пустых значений в столбцах

In [12]:
nan_count=100*data.isna().sum().sort_values(ascending=False)/data.shape[0]
fig=px.bar(x=nan_count.index,y=nan_count.values, labels={"y": "Nan ammount (%)","x": "Feature"})
fig.show()

Столбцы 'privat pool' и 'mis-id' имеют более чем 80% пропусков и вносят наименьший вклад в модель. Попробуем их удалить

In [13]:
data.drop('private pool', axis=1, inplace=True)
data.drop('mls-id', axis=1, inplace=True)

In [14]:
'''
X3 = data.drop('target',axis = 1).fillna(0)
y3 = data['target'].fillna(0)
skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X3, y3):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train3, X_test3 = X3[X3.index.isin(train_index)], X3[X3.index.isin(test_index)]
    y_train3, y_test3 = y3[y3.index.isin(train_index)], y3[y3.index.isin(test_index)]
    
#     # For training, fit() is used
    model2 = CatBoostRegressor(iterations=300)
    model2.fit(X_train3, y_train3,eval_set=(X_test3, y_test3), verbose=100, cat_features=['status', 'propertyType', 'street', 'baths', 'homeFacts', 'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'PrivatePool', 'MlsId'])


#     # For MAPE metric (or any other), we need the predictions of the model
    y_pred3 = model2.predict(X_test3)

    print(np.sqrt(mean_squared_error(y_test3, y_pred3)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test3, y_pred3))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test3, y_pred3))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test3, y_pred3))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")
'''

с удаленными двумя столбцами
The MAPE mertic for the default CatBoost model using 4-fold CV is: 14.33%.
The MAE mertic for default CatBoost model: 301005.62
The RMSE default CatBoost model: 1296315.00

Показания метрик улучшилось, столбцы можно удалить

In [15]:
data.info()

# EDA

# homeFacts

In [16]:
data['homeFacts']

In [17]:
data['homeFacts'][1]

На мой взгляд, самыми информативными здесь являются колонки с датой постройки и информацией о парковке. Информация о площади и цене в датасете уже есть. В колонках отопление и охлаждение много пропусков и сложно понять, как правильно обобщить эти данные, но если останется время поиграть можно. Пока не буду. 

In [18]:
type(data['homeFacts'])

In [19]:
type(data['homeFacts'][1])

Самое очевидное - попробовать вытащить информацию о годе постройки здания

In [20]:
ser = pd.Series(data['homeFacts']) 

ser.head(10) 

In [21]:
ser[0][35:39]

In [22]:
year = []
for j in range(0,377185):
    year.append(ser[j][35:39])

In [23]:
year[:10]

In [24]:
data['year'] = year

In [25]:
data['year'].value_counts().head(30)

In [26]:
data['year'].unique()

In [27]:
def year_cod(status):
    if "', '" in status: return '0'
    elif 'one,' in status: return '0'
    elif 'No D' in status: return '0'
    elif "1', " in status: return '0'
    else: 
        return status
    
data['year'] = data['year'].apply(lambda x: year_cod(x))
data['year'].value_counts()

Теперь проанализирую  данные о парковке

In [28]:
from ast import literal_eval
def funk(x):
    homefact_dict = literal_eval(x)
    homefact_lst = homefact_dict['atAGlanceFacts'][4].get('factValue')
    if homefact_lst:
        return str(homefact_lst)
    else:
        return np.nan
Parking = list(data[:].homeFacts.apply(funk))
Parking[:10]

In [29]:
data['Parking'] = Parking  

In [30]:
data['Parking'] = data['Parking'].apply(lambda x: str(x).lower()) # приведу данные к нижнему регистру

In [31]:
data['Parking'].value_counts().head(30)

In [32]:
def parking_cod(status):
    if '1' in status: return 'one'
    elif '2' in status: return 'two'
    elif '3' in status: return 'three'
    elif '4' in status: return 'four'
    elif 'attached' in status: return 'attached'
    elif 'detached' in status: return 'detached'
    elif 'carport' in status: return 'carport'
    elif 'no data' in status: return 'no info'
    elif '5' in status: return 'five'
    elif 'parking' in status: return 'parking'
    elif 'on street' in status: return 'on street'
    elif 'off street' in status: return 'off street'
    elif 'none' in status: return 'zero'
    elif '0' in status: return 'zero'
    elif 'garage' in status: return 'one'
    elif 'nan' in status: return 'no info'
    else: 
        return 'other'
    
data['Parking'] = data['Parking'].apply(lambda x: parking_cod(x))
data['Parking'].value_counts()

In [33]:
data.drop('homeFacts', axis=1, inplace=True)

In [34]:
data.info()

In [35]:
'''
X3 = data.drop('target',axis = 1).fillna(0)
y3 = data['target'].fillna(0)
skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X3, y3):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train3, X_test3 = X3[X3.index.isin(train_index)], X3[X3.index.isin(test_index)]
    y_train3, y_test3 = y3[y3.index.isin(train_index)], y3[y3.index.isin(test_index)]
    
#     # For training, fit() is used
    model2 = CatBoostRegressor(iterations=300)
    model2.fit(X_train3, y_train3,eval_set=(X_test3, y_test3), verbose=100, cat_features=['year','Parking','status','propertyType', 'street', 'baths',  'fireplace', 'city', 'schools', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'PrivatePool', 'MlsId'])


#     # For MAPE metric (or any other), we need the predictions of the model
    y_pred3 = model2.predict(X_test3)

    print(np.sqrt(mean_squared_error(y_test3, y_pred3)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test3, y_pred3))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test3, y_pred3))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test3, y_pred3))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")
'''

The MAPE mertic for the default CatBoost model using 4-fold CV is: 12.74%.
The MAE mertic for default CatBoost model: 300345.90
The RMSE default CatBoost model: 1303451.38

# schools

In [36]:
data['schools']

In [37]:
type(data['schools'])

In [38]:
data['schools'][1]

здесь наиболее интересными данными мне кажутся рейтинг школы и расстояние до школы. Попробую вычислить средний рейтинг и среднее расстояние.

In [39]:
def funk(x):
    Distance = literal_eval(x)
    Distance_list = Distance[0]['data'].get('Distance')
    if Distance_list:
        return Distance_list
    else:
        return '0'
    
Distance = list(data[:].schools.apply(funk))
Distance[:10]

In [40]:
data['Distance'] = Distance

In [41]:
data['Distance'][:10]

In [42]:
average_distance_school = []

In [43]:
for x in range(0,377185):
   
    l = data['Distance'][x]
    l1=[]
    for x in l:
        x =str(x).replace('mi','').replace('[','').replace(']','')
        l1.append(x)
    floatlist = [float(elem) for elem in l1]
    average_distance_school.append(sum(floatlist)/len(floatlist))
    
       

In [44]:
average_distance_school[:10]

In [45]:
len(average_distance_school)

In [46]:
data['average_distance_school'] = average_distance_school
data['average_distance_school'] = data['average_distance_school'].apply(lambda x: round(x,2))

In [47]:
data['average_distance_school'].value_counts().head(30)

In [48]:
data.average_distance_school.isna().sum()

In [49]:
def funk(x):
    rating = literal_eval(x)
    rating_list = rating[0].get('rating')
    if rating_list:
        return rating_list
    else:
        return '0'
    
rating_school = list(data[:].schools.apply(funk))
rating_school[:10]

In [50]:
data['rating_school'] = rating_school

In [51]:
data['rating_school'][:10]

In [52]:
 average_rating_school = []

In [53]:
for x in range(0,377185):
   
    m = data['rating_school'][x]
    m1=[]
    for x in m:
        x =str(x).replace('1/10','1').replace('2/10','2').replace('4/10','4').replace('5/10','5').replace('6/10','6').replace('7/10','7').replace('8/10','8').replace('9/10','9').replace('10/10','10').replace('None/10','0').replace('NA','0').replace('NR','0').replace('3/10','3').replace('[','').replace(']','').replace('','0')
        m1.append(x)
    floatlist = [float(elem) for elem in m1]
    average_rating_school.append((sum(floatlist)/len(floatlist))/10)

In [54]:
data['average_rating_school'] = average_rating_school
data['average_rating_school'] = data['average_rating_school'].apply(lambda x: round(x,2))

In [55]:
data['average_rating_school'].value_counts()

In [56]:
#data.nlargest(10, ['average_rating_school'])

In [57]:
data.info()

In [58]:
data.drop('schools', axis=1, inplace=True)
data.drop('Distance', axis=1, inplace=True)
data.drop('rating_school', axis=1, inplace=True)

In [59]:
'''
X3 = data.drop('target',axis = 1).fillna(0)
y3 = data['target'].fillna(0)
skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
cat_boost_mape_values = []
mae_values = []
rmse_values = []

for train_index, test_index in skf.split(X3, y3):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train3, X_test3 = X3[X3.index.isin(train_index)], X3[X3.index.isin(test_index)]
    y_train3, y_test3 = y3[y3.index.isin(train_index)], y3[y3.index.isin(test_index)]
    
#     # For training, fit() is used
    model2 = CatBoostRegressor(iterations=300)
    model2.fit(X_train3, y_train3,eval_set=(X_test3, y_test3), verbose=100, cat_features=['year','Parking','status','propertyType', 'street', 'baths',  'fireplace', 'city', 'sqft', 'zipcode', 'beds', 'state', 'stories', 'PrivatePool', 'MlsId'])


#     # For MAPE metric (or any other), we need the predictions of the model
    y_pred3 = model2.predict(X_test3)

    print(np.sqrt(mean_squared_error(y_test3, y_pred3)))

    cat_boost_mape_value = (mean_absolute_percentage_error(y_test3, y_pred3))
    cat_boost_mape_values.append(cat_boost_mape_value)
    print(cat_boost_mape_value)
    mae_value = (mean_absolute_error(y_test3, y_pred3))
    mae_values.append(mae_value)
    rmse_value = np.sqrt(mean_squared_error(y_test3, y_pred3))
    rmse_values.append(rmse_value)
    
print(f"The MAPE mertic for the default CatBoost model using 4-fold CV is: {(np.mean(cat_boost_mape_values)):0.2f}%.")
print(f"The MAE mertic for default CatBoost model: {(np.mean(mae_values)):0.2f}")
print(f"The RMSE default CatBoost model: {(np.mean(rmse_values)):0.2f}")
'''

The MAPE mertic for the default CatBoost model using 4-fold CV is: 12.66%.
The MAE mertic for default CatBoost model: 297139.37
The RMSE default CatBoost model: 1268556.49

In [60]:
data.head()

In [61]:
df = data.copy()

Несколько столбцов можно перевести из  object в int

In [62]:
df['average_rating_school'] = df['average_rating_school'].astype(int)
df['year'] = df['year'].astype(int)
df['target'] = df['target'].astype(int)
#df['sqft'] = df['sqft'].astype(int)
#df['zipcode'] = df['zipcode'].astype(int)

In [63]:
df.info()

Кодирую оставшиеся категориальные значения. Для столбцов city и state, содержащих очень много различных значений я пробовала One Hot Encoding, но при этом ноутбук вылетает из-за превышения размера памяти. 

In [64]:
df.status = df.status.astype('category').cat.codes
df.propertyType = df.propertyType.astype('category').cat.codes
df.street = df.street.astype('category').cat.codes
df.baths = df.baths.astype('category').cat.codes
df.fireplace = df.fireplace.astype('category').cat.codes
df.city = df.city.astype('category').cat.codes
df.sqft = df.sqft.astype('category').cat.codes
df.PrivatePool = df.PrivatePool.astype('category').cat.codes
df.beds = df.beds.astype('category').cat.codes
df.stories = df.stories.astype('category').cat.codes
df.state = df.state.astype('category').cat.codes
df.Parking = df.Parking.astype('category').cat.codes
df.MlsId = df.MlsId.astype('category').cat.codes
df.zipcode = df.zipcode.astype('category').cat.codes

In [65]:
'''
# Import label encoder 
from sklearn import preprocessing
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'.
data['status']= label_encoder.fit_transform(data['status']) 
data['propertyType']= label_encoder.fit_transform(data['propertyType']) 
data['street']= label_encoder.fit_transform(data['street'])
data['baths']= label_encoder.fit_transform(data['baths']) 
data['fireplace']= label_encoder.fit_transform(data['fireplace']) 
data['city']= label_encoder.fit_transform(data['city']) 
data['beds']= label_encoder.fit_transform(data['beds']) 
data['state']= label_encoder.fit_transform(data['state']) 
data['PrivatePool']= label_encoder.fit_transform(data['PrivatePool']) 
data['MlsId']= label_encoder.fit_transform(data['MlsId']) 
data['stories']= label_encoder.fit_transform(data['stories']) 
print(data.head())
'''

In [66]:
df.head()

In [67]:
#Выгрузим датасет для работы на локальной машине 
df.to_csv("/kaggle/working/df.csv")

# ML models

In [68]:
X = df.drop('target',axis = 1)
y = df['target']

Разбиваю данные на test и train

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [70]:
# пробовала стандартизировать числовые признаки. На бустинг моделях метрики остались такими же,  в RandomForest чуть хуже, а в ExtraTrees чуть лучше.
# Метрики StackingRegressor лучше со стандартизацией, поэтому буду ее использовать
num_cols=[]

num_cols += ['average_distance_school', 'year', 'average_rating_school']

scaler = MinMaxScaler()

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

## Model 1:  "Наивная" модель

В качестве нивной модели буду использовать линейную регрессию

In [71]:

naiv_model = LinearRegression()
naiv_model.fit(X_train, y_train)
naiv_predict = naiv_model.predict(X_test)
 

print(f"Точность наивной модели по метрике MAE: {(mean_absolute_error(y_test, naiv_predict)):0.2f}")
print(f"Точность наивной модели по метрике MAPE: {(mean_absolute_percentage_error(y_test, naiv_predict)):0.2f}%")
print(f"Точность наивной модели по метрике RMSE: {(np.sqrt(mean_squared_error(y_test, naiv_predict))):0.2f}")

# # попробую логарифмировние 
naiv_model_log = LinearRegression()
naiv_model_log.fit(X_train, np.log(y_train))
naiv_predict_log = np.exp(naiv_model_log.predict(X_test))


print(f"Точность наивной модели по метрике log MAE: {(mean_absolute_error(y_test, naiv_predict_log)):0.2f}")
print(f"Точность наивной модели по метрике log MAPE: {(mean_absolute_percentage_error(y_test, naiv_predict_log)):0.2f}%")
print(f"Точность наивной модели по метрике log RMSE: {(np.sqrt(mean_squared_error(y_test, naiv_predict_log))):0.2f}")

В дальнейшем, модели с логарифмированием целевой переменной показали лучший результат, поэтому out без логарифмирования показывать не буду

# Model 2 : Random Forest

In [72]:
#rf = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
#rf.fit(X_train, y_train)
#predict_rf = rf.predict(X_test)

#print(f"The MAPE mertics of the Random Forest model is: {(mean_absolute_percentage_error(y_test, predict_rf)):0.2f}%.")


rf_log = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rf_log.predict(X_test))
predict_rf_x = np.exp(rf_log.predict(X_train))

print(f"The MAE train Random Forest model is : {(mean_absolute_error(y_train, predict_rf_x)):0.2f}.")
print(f"The MAE test  Random Forest model is : {(mean_absolute_error(y_test, predict_rf_log)):0.2f}.")

print(f"The MAPE train Random Forest model is : {(mean_absolute_percentage_error(y_train, predict_rf_x)):0.2f}%.")
print(f"The MAPE test  Random Forest model is : {(mean_absolute_percentage_error(y_test, predict_rf_log)):0.2f}%.")

print(f"The RMSE train Random Forest model is : {(np.sqrt(mean_squared_error(y_train, predict_rf_x))):0.2f}.")
print(f"The RMSE test Random Forest model is : {(np.sqrt(mean_squared_error(y_test, predict_rf_log))):0.2f}.")

The MAE mertic for the log Random Forest model is : 219079.62

The MAPE mertic for the log Random Forest model is : 2.33%.

The RMSE mertic for the log Random Forest model is : 1490508.55

# Model 3 : ExtraTreesRegressor

In [73]:
#etr = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
#etr.fit(X_train, y_train)
#predict_etr = etr.predict(X_test)

#print(f"The MAPE mertics of the ExtraTrees model is: {(mean_absolute_percentage_error(y_test, predict_etr)):0.2f}%.")


etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_log.fit(X_train, np.log(y_train))
predict_etr_log = np.exp(etr_log.predict(X_test))
predict_etr_x = np.exp(etr_log.predict(X_train))

print(f"The MAE train ExtraTrees model is : {(mean_absolute_error(y_train, predict_etr_x)):0.2f}.")
print(f"The MAE test ExtraTrees model is : {(mean_absolute_error(y_test, predict_etr_log)):0.2f}.")

print(f"The MAPE train ExtraTrees model is : {(mean_absolute_percentage_error(y_train, predict_etr_x)):0.2f}%.")
print(f"The MAPE test ExtraTrees model is : {(mean_absolute_percentage_error(y_test, predict_etr_log)):0.2f}%.")

print(f"The RMSE train ExtraTrees model is : {(np.sqrt(mean_squared_error(y_train, predict_etr_x))):0.2f}.")
print(f"The RMSE test ExtraTrees model is : {(np.sqrt(mean_squared_error(y_test, predict_etr_log))):0.2f}.")

The MAE mertic for the log ExtraTrees model is : 223392.76

The MAPE mertic for the log ExtraTrees model is : 3.86%.

The RMSE mertic for the log ExtraTrees model is : 1456802.16

# Model 4 : CatBoost

In [74]:
#cat_boost = CatBoostRegressor(iterations = 10000,
#                          random_seed = RANDOM_SEED,
#                          eval_metric='MAPE',
#                          custom_metric=['R2', 'MAE'],
#                          silent=True,
#                         )
#cat_boost.fit(X_train, y_train,
#          eval_set=(X_test, y_test),
#          verbose_eval=0,
#          use_best_model=True,
#          )

#cat_boost.save_model('catboost_single_model_baseline.model')


#predict_cat_boost = cat_boost.predict(X_test)


#print(f"The MAPE mertic for the default CatBoost model: {(mean_absolute_percentage_error(y_test, predict_cat_boost)):0.2f}%")

The MAPE mertic for the default CatBoost model: 3.16%

In [75]:
log_cat_boost = CatBoostRegressor(iterations = 10000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
log_cat_boost.fit(X_train, np.log(y_train),
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=0,
          use_best_model=True,
          )

log_cat_boost.save_model('catboost_single_model_baseline.model')


log_predict_cat_boost = log_cat_boost.predict(X_test)
predict_cat_boost_x = np.exp(log_cat_boost.predict(X_train))

print(f"The MAE train CatBoost model: {(mean_absolute_error(y_train, predict_cat_boost_x)):0.2f}.")
print(f"The MAE test CatBoost model: {(mean_absolute_error(y_test, log_predict_cat_boost)):0.2f}")

print(f"The MAPE train CatBoost model: {(mean_absolute_percentage_error(y_train, predict_cat_boost_x)):0.2f}%.")
print(f"The MAPE test CatBoost model: {(mean_absolute_percentage_error(y_test, log_predict_cat_boost)):0.2f}%")

print(f"The RMSE train CatBoost model: {(np.sqrt(mean_squared_error(y_train, predict_cat_boost_x))):0.2f}.")
print(f"The RMSE test CatBoost model: {(np.sqrt(mean_squared_error(y_test, log_predict_cat_boost))):0.2f}")

The MAE mertic for the CatBoost model: 644162.03

The MAPE mertic for the CatBoost model: 1.00%

The RMSE mertic for the CatBoost model: 2050654.88

# Model 5: GradientBoosting

In [76]:
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, np.log(y_train))
predict_gb = gb.predict(X_test)
predict_gb_x = gb.predict(X_train)

print(f"The MAE train GradientBoosting model: {(mean_absolute_error(y_train, predict_gb_x)):0.2f}.")
print(f"The MAE test GradientBoosting model: {(mean_absolute_error(y_test, predict_gb)):0.2f}")

print(f"The MAPE train GradientBoosting model: {(mean_absolute_percentage_error(y_train, predict_gb_x)):0.2f}%.")
print(f"The MAPE test GradientBoosting model: {(mean_absolute_percentage_error(y_test, predict_gb)):0.2f}%")

print(f"The RMSE train GradientBoosting model: {(np.sqrt(mean_squared_error(y_train, predict_gb_x))):0.2f}.")
print(f"The RMSE test GradientBoosting model: {(np.sqrt(mean_squared_error(y_test, predict_gb))):0.2f}")

The MAE mertic for GradientBoosting model: 644162.03

The MAPE mertic for GradientBoosting model: 1.00%

The RMSE mertic for GradientBoosting model: 2050655.03

# Model 6 : StackingRegressor

Попробую комбинацию из моделей, показавших лучшие метрики

In [77]:
estimators = [
     ('cat_boost', CatBoostRegressor(iterations = 10000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True
                         )),
     ('gb', GradientBoostingRegressor(random_state=RANDOM_SEED))
]

sr_log = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
)

sr_log.fit(X_train, np.log(y_train))

y_pred = np.exp(sr_log.predict(X_test))

print(f"The MAE mertic for StackingRegressor model: {(mean_absolute_error(y_test, y_pred)):0.2f}")
print(f"The MAPE mertic for StackingRegressor model: {(mean_absolute_percentage_error(y_test, y_pred)):0.2f}%")
print(f"The RMSE mertic for StackingRegressor model: {(np.sqrt(mean_squared_error(y_test, y_pred))):0.2f}")

The MAE mertic for the default StackingRegressor model: 256430.26

The MAPE mertic for the default StackingRegressor model: 2.83%

The RMSE mertic for the default StackingRegressor model: 1462428.77

# Итог

В ходе выполнения проекта:

Загружены данные, произведена их обработка;

Заполнены пропуски;

Очищены nan;

Созданы новые признаки на основе имеющихся;

Построены различные модели классического ML для решения задачи регрессии;

Для каждой построенной модели получены соответствующие метрики;

Построена сводная модель, которая улучшила значение показателей и предотвратила переобучение.


В результате финальные метрики:

MAE  256430.26

MAPE 2.83%

RMSE  1462428.77